In [17]:
#Schema at https://dbdiagram.io/d/63223c6e0911f91ba5b12259

In [18]:
# Importing sqlalchemy
import sqlalchemy as sql

In [19]:
# Some more imports.
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [20]:
Base = declarative_base()

class Pokemons(Base):
    __tablename__ = 'pokemons'
    id = Column(Integer, primary_key=True)
    pokemon_name = Column(String(50), nullable=False)
    total = Column(Integer, nullable=False)
    hp = Column(Integer, nullable=False)
    attack = Column(Integer, nullable=False)
    defense = Column(Integer, nullable=False)
    sp_attack = Column(Integer, nullable=False)
    sp_defense = Column(Integer, nullable=False)
    speed = Column(Integer, nullable=False)
    legendary = Column(Boolean, nullable=False)

    type_1 = Column(Integer, ForeignKey('types.id'))
    type_2 = Column(Integer, ForeignKey('types.id'))
    generation = Column(Integer,ForeignKey('generations.id'), nullable=False)

    # I DON'T KNOW WHAT THESE DO =/
    # types = relationship("Types", back_populates="pokemons")
    # generations = relationship("Generations", back_populates="pokemons")

class Generations(Base):
    __tablename__ = 'generations'
    id = Column(Integer, primary_key=True)
    generation = Column(Integer, nullable=False)
    #I am not sure about these....
    # pokemons = relationship("Pokemons", back_populates="generations")

class Types(Base):
    __tablename__ = 'types'
    id = Column(Integer, primary_key=True)
    type_name = Column(String(50), nullable=False)
    #I am not sure about these....
    # pokemons = relationship("Pokemons", back_populates="types")


# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///pokemon_shop.db')

# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)

In [21]:
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

In [22]:
#Here we move csv data to our new database.
import csv
filename = 'Pokemons.csv'
#Step One. Retrieve types and generations.
types = []
generations = []

with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        #First row is a header with column names. Skip it
        if row[0] == "#":
            continue
        #Grab type 1 and 2 from a row
        type_1 = row[2]
        type_2 = row[3]
        #Grab generation and transforming it into a number. Add it to list.
        generation = int(row[11])
        generations.append(generation)
        #Because we need unique types - check if the type is already on the list.
        if type_1 not in types:
            types.append(type_1)
        else: continue
        
        if type_2 not in types:
            types.append(type_2)
        else: continue
#Add types and generations to their tables.       
for i in types:
    new_type = Types(type_name=i)
    session.add(new_type)
    session.commit()
for i in list(set(generations)):
    new_generation = Generations(generation=i)
    session.add(new_generation)
    session.commit()

In [23]:
#Step Two. Retrieve Pokemons.
with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        #Skip header.
        if row[0] == "#":
            continue
        #Grab values. Transform strings to numbers where needed.
        name = row[1]
        type_1 = row[2]
        type_2 = row[3]
        #Because for this table we need foreing keys as numbers - grab them from the list above. Add 1 coz 0 is 1 xD
        type_1_index = types.index(type_1)+1
        type_2_index = types.index(type_2)+1
        total = int(row[4])
        hp = int(row[5])
        attack = int(row[6])
        defense = int(row[7])
        sp_attack = int(row[8])
        sp_defense = int(row[9])
        speed = int(row[10])
        generation = int(row[11])
        #Ternary operator for legendary parameter.
        legendary = False if row[12] == "False" else True
        #Create new pokemon and add it to the database.
        new_pokemon = Pokemons(
            pokemon_name = name,
            total = total,
            hp = hp,
            attack = attack,
            defense = defense,
            sp_attack = sp_attack,
            sp_defense = sp_defense,
            speed = speed,
            legendary = legendary,
            #Indexes go here not values
            type_1 = type_1_index,
            type_2 = type_2_index,
            generation = generation
        )
        session.add(new_pokemon)
        session.commit() 

In [24]:
#Print all pokemons.
s = sql.select([Pokemons])
rp = session.execute(s)
results = rp.fetchall()
for i in results:
    print(i.Pokemons.pokemon_name, i.Pokemons.type_1, i.Pokemons.type_2, i.Pokemons.total,i.Pokemons.hp, i.Pokemons.attack, i.Pokemons.defense, i.Pokemons.sp_attack, i.Pokemons.sp_defense, i.Pokemons.speed, i.Pokemons.generation, i.Pokemons.legendary)

Bulbasaur 1 2 318 45 49 49 65 65 45 1 False
Ivysaur 1 2 405 60 62 63 80 80 60 1 False
Venusaur 1 2 525 80 82 83 100 100 80 1 False
VenusaurMega Venusaur 1 2 625 80 100 123 122 120 80 1 False
Charmander 3 4 309 39 52 43 60 50 65 1 False
Charmeleon 3 4 405 58 64 58 80 65 80 1 False
Charizard 3 8 534 78 84 78 109 85 100 1 False
CharizardMega Charizard X 3 17 634 78 130 111 130 85 100 1 False
CharizardMega Charizard Y 3 8 634 78 104 78 159 115 100 1 False
Squirtle 5 4 314 44 48 65 50 64 43 1 False
Wartortle 5 4 405 59 63 80 65 80 58 1 False
Blastoise 5 4 530 79 83 100 85 105 78 1 False
BlastoiseMega Blastoise 5 4 630 79 103 120 135 115 78 1 False
Caterpie 6 4 195 45 30 35 20 20 45 1 False
Metapod 6 4 205 50 20 55 25 25 30 1 False
Butterfree 6 8 395 60 45 50 90 80 70 1 False
Weedle 6 2 195 40 35 30 20 20 50 1 False
Kakuna 6 2 205 45 25 50 25 25 35 1 False
Beedrill 6 2 395 65 90 40 45 80 75 1 False
BeedrillMega Beedrill 6 2 495 65 150 40 15 80 145 1 False
Pidgey 7 8 251 40 45 40 35 35 56 1 F

In [25]:
#Print all types with indexes.
s = sql.select([Types])
rp = session.execute(s)
results = rp.fetchall()

for i in results:
    print(i.Types.type_name, i.Types.id)

Grass 1
Poison 2
Fire 3
 4
Water 5
Bug 6
Normal 7
Flying 8
Electric 9
Ground 10
Fairy 11
Fighting 12
Psychic 13
Rock 14
Ghost 15
Ice 16
Dragon 17
Dark 18
Steel 19


In [26]:
#Print all generations with indexes.
s = sql.select([Generations])
rp = session.execute(s)
results = rp.fetchall()
for i in results:
    print(i.Generations.generation, i.Generations.id)


1 1
2 2
3 3
4 4
5 5
6 6


In [27]:



# Example with a where statement.
s = sql.select([Generations]).where(Generations.id == 1)
rp = session.execute(s)
results = rp.fetchall()
results

s = "SELECT * FROM pokemons"
rp = session.execute(s)
results = rp.fetchall()
results

s = "SELECT pokemon_name, attack, type_name FROM pokemons, types WHERE types.id = pokemons.type_1 GROUP BY attack"
rp = session.execute(s)
results = rp.fetchall()
results

[('Chansey', 5, 'Normal'),
 ('Magikarp', 10, 'Water'),
 ('Feebas', 15, 'Water'),
 ('Metapod', 20, 'Bug'),
 ('Spewpa', 22, 'Bug'),
 ('Wynaut', 23, 'Psychic'),
 ('Bronzor', 24, 'Steel'),
 ('Kakuna', 25, 'Bug'),
 ('Cottonee', 27, 'Grass'),
 ('Burmy', 29, 'Bug'),
 ('Caterpie', 30, 'Bug'),
 ('Wobbuffet', 33, 'Psychic'),
 ('Weedle', 35, 'Bug'),
 ('Bunnelby', 36, 'Normal'),
 ('Chinchou', 38, 'Water'),
 ('Tentacool', 40, 'Water'),
 ('Vulpix', 41, 'Fire'),
 ('Shieldon', 42, 'Rock'),
 ('Gulpin', 43, 'Poison'),
 ('Ducklett', 44, 'Water'),
 ('Butterfree', 45, 'Bug'),
 ('Sentret', 46, 'Normal'),
 ('Nidoran♀', 47, 'Poison'),
 ('Squirtle', 48, 'Water'),
 ('Bulbasaur', 49, 'Grass'),
 ('Oddish', 50, 'Grass'),
 ('Whismur', 51, 'Normal'),
 ('Charmander', 52, 'Fire'),
 ('Pansage', 53, 'Grass'),
 ('Inkay', 54, 'Dark'),
 ('Pikachu', 55, 'Electric'),
 ('Rattata', 56, 'Normal'),
 ('Nidoran♂', 57, 'Poison'),
 ('Lanturn', 58, 'Water'),
 ('WormadamPlant Cloak', 59, 'Bug'),
 ('Pidgeotto', 60, 'Normal'),
 ('Croagu

In [28]:
# Notice anything different about the query?
s = session.query(Pokemons)
s = s.filter(Pokemons.id == 1)

# Update example.
s.update({Pokemons.hp: 45})
session.commit()

In [35]:
#Add one super strong pokemon:
super_pokemon = Pokemons(
            pokemon_name = "Tyranauseras X 3000",
            total = 10000,
            hp = 500,
            attack = 500,
            defense = 500,
            sp_attack = 500,
            sp_defense = 500,
            speed = 500,
            legendary = True,
            #Indexes go here not values
            type_1 = 13,
            type_2 = 10,
            generation = 5
        )
session.add(super_pokemon)
session.commit() 

s = session.query(Pokemons)
s = s.filter(Pokemons.attack == 500)
rp = session.execute(s)
results = rp.fetchall()
for i in results:
    print(i.Pokemons.id, i.Pokemons.pokemon_name, i.Pokemons.type_1, i.Pokemons.type_2, i.Pokemons.total,i.Pokemons.hp, i.Pokemons.attack, i.Pokemons.defense, i.Pokemons.sp_attack, i.Pokemons.sp_defense, i.Pokemons.speed, i.Pokemons.generation, i.Pokemons.legendary)

/var/folders/hp/p4nb_ltj30752cm9cfz3pd6m0000gn/T/ipykernel_32310/174661995.py:22: SADeprecationWarning: Object <sqlalchemy.orm.query.Query object at 0x7f9c7838ef40> should not be used directly in a SQL statement context, such as passing to methods such as session.execute().  This usage will be disallowed in a future release.  Please use Core select() / update() / delete() etc. with Session.execute() and other statement execution methods.
  rp = session.execute(s)


In [30]:
# Delete example
s = sql.delete(Pokemons).where(Pokemons.id >= 800)
rp = session.execute(s)

In [31]:
# Closing session.
session.close()